# Random Walks in Higher-Order Networks


[Run notebook in Google Colab](https://colab.research.google.com/github/pathpy/pathpy/blob/master/doc/tutorial/higher_order_random_walks.ipynb)  
[Download notebook](https://github.com/pathpy/pathpy/raw/master/doc/tutorial/higher_order_random_walks.ipynb)

In [1]:
import pathpy as pp
from pprint import pprint

We can use `pathpy` to simulate random walks on higher-order networks. Let us first create a simple Higher-Order Network:

In [12]:
n = pp.Network(directed=False)
n.add_edge('a', 'b', weight=1, uid='a-b')
n.add_edge('b', 'c', weight=1, uid='b-c')
n.add_edge('c', 'a', weight=2, uid='c-a')
n.add_edge('c', 'd', weight=1, uid='c-d')
n.add_edge('d', 'a', weight=1, uid='d-a')

n.plot()

In [14]:
n2 = pp.HigherOrderNetwork()
v1 = pp.HigherOrderNode(n.edges['a-b'], uid='a-b')
v2 = pp.HigherOrderNode(n.edges['b-c'], uid='b-c')
v3 = pp.HigherOrderNode(n.edges['c-a'], uid='c-a')
v4 = pp.HigherOrderNode(n.edges['c-d'], uid='c-d')
v5 = pp.HigherOrderNode(n.edges['d-a'], uid='d-a')

n2.add_edge(v1, v2, uid='a-b-c', weight=1)
n2.add_edge(v2, v3, uid='b-c-a', weight=1)
n2.add_edge(v2, v4, uid='b-c-d', weight=0.2)
n2.add_edge(v3, v1, uid='c-a-b', weight=1)
n2.add_edge(v4, v5, uid='c-d-a', weight=0.2)
n2.add_edge(v5, v1, uid='d-a-b', weight=1)
n2.plot()

To simulate a random in the higher-order network, we can use the same class as for a standard, first-order network:

In [3]:
rw2 = pp.processes.RandomWalk(n2, weight='weight')

We can inspect the transition matrix as follows:

In [4]:
print(rw2.transition_matrix_pd())

     a-b  b-c       c-a       c-d  d-a
a-b  0.0  1.0  0.000000  0.000000  0.0
b-c  0.0  0.0  0.833333  0.166667  0.0
c-a  1.0  0.0  0.000000  0.000000  0.0
c-d  0.0  0.0  0.000000  0.000000  1.0
d-a  1.0  0.0  0.000000  0.000000  0.0


We can use the following code to simulate a random walk in the higher-order network. The generated `Path` object is automatically mapped to the first-order nodes. We have to specify a second-order node as the start node (or the number of randomly chosen second-order nodes). 

Based on the higher-order network above, a transition from `b` via `c` to `d` can never occur:

In [5]:
data = rw2.run_experiment(steps=50, runs=['b-c', 'c-d'])
print(data)

     run_id seed  time node  state
0         0  b-c     0  d-a  False
1         0  b-c     0  c-d  False
2         0  b-c     0  b-c   True
3         0  b-c     0  c-a  False
4         0  b-c     0  a-b  False
..      ...  ...   ...  ...    ...
205       1  c-d    48  a-b  False
206       1  c-d    49  c-a   True
207       1  c-d    49  b-c  False
208       1  c-d    50  a-b   True
209       1  c-d    50  c-a  False

[210 rows x 5 columns]


In [6]:
rw2.plot(data)

Conversely, the transition from `b` via `c` to `d` will occur in the corresponding first-order random walk:

In [7]:
p = rw2.get_path(data)
print([ v.uid for v in p.nodes ])

['b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'd', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'd', 'a', 'b', 'c', 'a', 'b', 'c', 'd', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b']


In [8]:
p = rw2.get_path(data, first_order=False)
print([ v.uid for v in p.nodes ])

['b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-d', 'd-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-d', 'd-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-d', 'd-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b']


In [9]:
pc = rw2.get_paths(data)
for p in pc:
    	print([ v.uid for v in p.nodes ])

['b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'd', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'd', 'a', 'b', 'c', 'a', 'b', 'c', 'd', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b']
['c', 'd', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'd', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'd', 'a', 'b', 'c', 'd', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b', 'c', 'a', 'b']


In [10]:
pc = rw2.get_paths(data, first_order=False)
for p in pc:
    	print([ v.uid for v in p.nodes ])

['b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-d', 'd-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-d', 'd-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-d', 'd-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b']
['c-d', 'd-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-d', 'd-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-d', 'd-a', 'a-b', 'b-c', 'c-d', 'd-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b', 'b-c', 'c-a', 'a-b']


In [11]:
rw2.plot_first_order(data, n)